<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
import numpy as np
import pandas as pd

In [0]:
df = pd.read_fwf('https://www.gutenberg.org/files/100/100-0.txt')

In [0]:
df = df.reset_index()

In [0]:
df = df.drop('Unnamed: 0', axis = 1)

In [0]:
df = df.iloc[134:166476]

In [0]:
df = df.dropna().reset_index().drop('level_0', axis=1)

In [28]:
df.head()

,index
0,THE SONNETS
1,1
2,"From fairest creatures we desire increase,"
3,"That thereby beauty’s rose might never die,"
4,"But as the riper should by time decease,"


In [29]:
df.tail(10)

,index
138923,My throbbing heart shall rock thee day and night:
138924,There shall not be one minute in an hour
138925,Wherein I will not kiss my sweet love’s flower.”
138926,"Thus weary of the world, away she hies, ..."
138927,And yokes her silver doves; by whose swift aid
138928,"Their mistress mounted through the empty skies,"
138929,In her light chariot quickly is convey’d; ...
138930,"Holding their course to Paphos, where their queen"
138931,Means to immure herself and not be seen.
138932,FINIS


In [0]:
a = int(138932/2)

In [31]:
sample = df.iloc[0:a]
sample.tail()

,index
69461,"Come, go we to the King. Our power is ready;"
69462,Our lack is nothing but our leave. Macbeth
69463,"Is ripe for shaking, and the powers above"
69464,Put on their instruments. Receive what cheer y...
69465,The night is long that never finds the day.


In [0]:
from tensorflow.keras.callbacks import LambdaCallback, EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing import sequence

import sys
import random

In [0]:
data = sample['index'].tolist()

In [0]:
# convert list of strings to a string
text = " ".join(data)

#create set of unique characters
chars = list(set(text))

# look up tables
char_int = {c:i for i, c in enumerate(chars)}
int_char = {i:c for i, c in enumerate(chars)}

In [35]:
# create sequence data
maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = []
next_char = []

for i in range(0, len(encoded) -maxlen, step):

  sequences.append(encoded[i : i + maxlen])
  next_char.append(encoded[i + maxlen])

print(f'Sequences: {len(sequences)}')

Sequences: 538906


In [0]:
# create x and y
x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
  for t, char in enumerate(sequence):
    x[i,t,char] = 1

  y[i, next_char[i]] = 1

In [37]:
x.shape

(538906, 40, 97)

In [38]:
y.shape

(538906, 97)

In [0]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
def sample(preds):
  preds = np.asarray(preds).astype('float64')
  preds = np.log(preds) / 1
  exp_preds = np.exp(preds)
  preds = exp_preds / np.sum(exp_preds)
  probas = np.random.multinomial(1, preds, 1)
  return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, _):
  #ffunction printed at the end of each epoch. Prints generated text.

  print(f'\n-----Generated text after Epoch {epoch}')

  start_index = random.randint(0, len(text) - maxlen -1)

  generated = ''

  sentence = text[start_index: start_index + maxlen]
  generated += sentence

  print(f'----- Generated with seed: {sentence}')
  sys.stdout.write(generated)

  for i in range(400):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds)
    next_char = int_char[next_index]

    sentence = sentence[1:] + next_char

    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [43]:
model.fit(x, y,
          batch_size=32, #change to < 32 after sample works
          epochs=100,
          validation_split=.2,
          callbacks=[print_callback])

Epoch 1/100
13470/13473 [============================>.] - ETA: 0s - loss: 1.7209
-----Generated text after Epoch 0
----- Generated with seed: ne; and he most violent author Of his ow
ne; and he most violent author Of his owatly like. MENALIOS. I ab reating them, his infoed bode wilbazeth, and congainst you a bast dast insthue I am. TAATSS. Why, I ’tis frought their of I wilpot'd; Neemed thou did wouth here, wearture. Cosinte, deat is owh utreives shay beffeet en inother wither of many time; and made that]  THath we had sake out mount To dettens all buht; welles,  the goke can beated! I tank, and have he had ratche a
13473/13473 [==============================] - 201s 15ms/step - loss: 1.7209 - val_loss: 1.8229
Epoch 2/100
13472/13473 [============================>.] - ETA: 0s - loss: 1.6775
-----Generated text after Epoch 1
----- Generated with seed: st made me traitor to myself. But lest m
st made me traitor to myself. But lest makely! [_Exit Pail _ARIONACHATHOM VALOV EYORK once, and

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


espruions parts Undeed HAMLET. Ibolvery! Whee warth shall togetionfed true, would make your lordship or delled Hercted, law; nor you, bring servite is that. ExUpun, Which horsestref, Took to
13473/13473 [==============================] - 200s 15ms/step - loss: 1.4790 - val_loss: 1.7597
Epoch 17/100
13473/13473 [==============================] - ETA: 0s - loss: 1.4731
-----Generated text after Epoch 16
----- Generated with seed: s form and with this addition, in French
s form and with this addition, in French nor cruestic. Coint is to them to out of divers with wit. King set in the cullementle fair trymainess, Thou seas a hot sound have here ambeed, Did I sea Whreact, fortuble facsion, to our sime, and KING EDWARD. be asseath speak'sh, your sentret: it well indeel, wilt but I keed To’t take for thought you with thee frienachal and go to forg'd, Whose offich or his ear master, and SpEintage. Though. AN
13473/13473 [==============================] - 200s 15ms/step - loss: 1.4731 - val_los

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN